# Practice Questions - 08-29-2025

In [1]:
import os
import sqlite3
import pandas as pd

In [2]:
os.getcwd()

'/Users/neilbardhan/Documents/python-projects/technical-interview-prep/data/notebooks'

In [3]:
os.chdir('..')

## Connect to Database

In [4]:
def print_tables(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(tables)
    cursor.close()

In [5]:
DB_NAME = 'ecommerce.db'
DB_PATH = "{}/{}".format(os.getcwd(), "datasets/")
db_file_path = "{}{}".format(DB_PATH, DB_NAME)

In [6]:
if os.path.isfile(db_file_path):
    connection = sqlite3.connect(db_file_path)
    print_tables(connection)

[('customer',), ('product',), ('transactions',), ('transaction_items',), ('reviews',)]


### Print Schemas of Tables

In [7]:
def get_table_schema(connection, table_name):
    pragma_query = "PRAGMA table_info({});".format(table_name)
    print(pragma_query)
    cursor = connection.cursor()
    cursor.execute(pragma_query)
    pragma_info = cursor.fetchall()
    for row in pragma_info:
        print("{}:\t{}".format(row[1], row[2]))
    cursor.close()

In [8]:
get_table_schema(connection, "customer")

PRAGMA table_info(customer);
customer_id:	INTEGER
name:	TEXT
email:	TEXT
signup_date:	TEXT
country:	TEXT


In [9]:
get_table_schema(connection, "product")

PRAGMA table_info(product);
product_id:	INTEGER
name:	TEXT
category:	TEXT
price:	REAL


In [10]:
get_table_schema(connection, "transactions")

PRAGMA table_info(transactions);
order_id:	INTEGER
customer_id:	INTEGER
order_date:	TEXT
total_amount:	REAL


In [11]:
get_table_schema(connection, "transaction_items")

PRAGMA table_info(transaction_items);
order_item_id:	INTEGER
order_id:	INTEGER
product_id:	INTEGER
quantity:	INTEGER
item_price:	REAL


In [12]:
get_table_schema(connection, "reviews")

PRAGMA table_info(reviews);
review_id:	INTEGER
order_id:	INTEGER
product_id:	INTEGER
rating:	INTEGER
review_date:	TEXT


## Practice Questions

In [13]:
def query_runner(connection, query):
    query_df = pd.read_sql_query(query, connection)
    return query_df

In [14]:
customer_table_all_query = """
SELECT *
FROM customer;
"""
query_results_df = query_runner(connection, customer_table_all_query)
query_results_df.head()

,customer_id,name,email,signup_date,country
0,1,Raymond Conley,vking@example.com,2025-03-14,Malaysia
1,2,Robert Andersen,crystal34@example.org,2024-03-14,Niger
2,3,Jody Martinez,jlane@example.com,2023-12-15,Jamaica
3,4,Kenneth Shepherd,kimberlypeters@example.net,2023-07-25,Peru
4,5,Carolyn Taylor,joshua08@example.org,2025-03-13,Mayotte


### List the first 10 customers who signed up in 2022. Show `customer_id, name, signup_date`.

In [23]:
sql_query = """
SELECT customer_id, name, signup_date
FROM customer
WHERE strftime('%Y', signup_date) = '2022'
ORDER BY signup_date ASC
LIMIT 10;
"""
query_results_df = query_runner(connection, sql_query)
print(query_results_df.shape)
query_results_df.head(10)

(10, 3)


,customer_id,name,signup_date
0,161,Jennifer Collins,2022-08-27
1,437,Brenda Wheeler,2022-08-27
2,416,Charles Thompson,2022-08-28
3,465,Ashley Zimmerman,2022-08-29
4,364,April Merritt,2022-08-30
5,546,Dawn Rodriguez,2022-08-30
6,563,Debra Morgan,2022-08-30
7,513,Tiffany Shaw,2022-09-01
8,509,Raymond Rose,2022-09-02
9,669,Rhonda Richardson,2022-09-05


### Find the total number of orders and the average order value (AOV) across all customers.

In [29]:
sql_query = """
SELECT COUNT(*) as total_orders, ROUND(AVG(total_amount), 2) as avg_order_value
FROM transactions;
"""
query_results_df = query_runner(connection, sql_query)
print(query_results_df.shape)
query_results_df.head(10)

(1, 2)


,total_orders,avg_order_value
0,4162,1456.85


### Find the total number of orders and the average order value (AOV) _per_ customer.

In [31]:
sql_query = """
SELECT customer_id, COUNT(*) as total_orders, ROUND(SUM(total_amount)/COUNT(*), 2) as aov
FROM transactions
GROUP BY customer_id
ORDER BY aov DESC;
"""
query_results_df = query_runner(connection, sql_query)
print(query_results_df.shape)
query_results_df.head(10)

(750, 3)


,customer_id,total_orders,aov
0,673,1,3965.48
1,199,1,3584.72
2,592,3,3584.59
3,107,3,3159.61
4,657,2,3128.51
5,564,1,3070.76
6,210,1,2960.79
7,645,1,2930.01
8,65,2,2911.02
9,400,1,2909.12


### Find the top 5 products by total sales revenue (quantity × item_price). Show `product_id`, `product_name`, and `total revenue`.

In [41]:
sql_query = """
SELECT p.product_id, name, item_price * SUM(quantity) AS total_sales_revenue
FROM product p INNER JOIN transaction_items t
ON p.product_id = t.product_id
GROUP BY p.product_id
ORDER BY total_sales_revenue DESC
LIMIT 5;
"""
query_results_df = query_runner(connection, sql_query)
print(query_results_df.shape)
query_results_df.head(10)

(5, 3)


,product_id,name,total_sales_revenue
0,132,Which,83301.27
1,3,Teacher,72025.50
2,18,Dream,71404.22
3,153,Family,68812.17
4,59,Act,66737.10


### For each country, calculate the average lifetime spend per customer.

Lifetime spend = sum of all orders for that customer.

In [48]:
sql_query = """
WITH lifetime_spend_per_customer AS (
SELECT customer_id, SUM(total_amount) AS total_spend
FROM transactions
GROUP BY customer_id
)
SELECT c.country, AVG(l.total_spend) AS avg_lifetime_spend_per_customer
FROM lifetime_spend_per_customer l INNER JOIN customer c
ON l.customer_id = c.customer_id
GROUP BY c.country
ORDER BY avg_lifetime_spend_per_customer DESC;
"""
query_results_df = query_runner(connection, sql_query)
print(query_results_df.shape)
query_results_df.head(10)

(235, 2)


,country,avg_lifetime_spend_per_customer
0,Aruba,17339.590
1,Paraguay,16201.340
2,Kiribati,15242.920
3,Germany,14601.815
4,Croatia,14344.400
5,French Southern Territories,14027.510
6,Bouvet Island (Bouvetoya),13225.255
7,Ghana,12786.520
8,Guernsey,12719.350
9,Venezuela,12670.705


### For each customer, rank their orders by `order_date` (most recent first). Show `customer_id`, `order_id`, `order_date`, and the `rank`.

In [53]:
sql_query = """
SELECT customer_id, order_id, order_date, RANK() OVER (PARTITION BY customer_id ORDER BY order_date DESC) AS rank
FROM transactions;
"""
query_results_df = query_runner(connection, sql_query)
print(query_results_df.shape)
query_results_df.head(10)

(4162, 4)


,customer_id,order_id,order_date,rank
0,1,1,2024-09-26,1
1,2,2,2025-06-26,1
2,2,9,2025-05-08,2
3,2,8,2025-05-07,3
4,2,3,2024-12-10,4
5,2,7,2024-05-06,5
6,2,6,2023-10-23,6
7,2,10,2023-07-25,7
8,2,5,2023-03-09,8
9,2,4,2022-09-10,9


  - `RANK()`: Will skip ranks if there are ties.
  - `ROW_NUMBER()` is often preferred if you want a strict sequence (1, 2, 3 …).
  - `DENSE_RANK()` avoids gaps but keeps ties.